## Setup

In [36]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [37]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# from tensorflow.keras.layers import TextVectorization
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm

## Downloading the data

In [2]:
# TODO in colab (last)

## Parsing the data

We prepend the token "[start]" and we append the token "[end]" to each Romanian sentence.

In [3]:
with open('europarl-v7.ro-en.en') as f:
    en_lines = f.read().split("\n")[:-1]
with open('europarl-v7.ro-en.ro') as f:
    ro_lines = f.read().split("\n")[:-1]    
    
text_pairs = []
assert len(en_lines) == len(ro_lines)
for i in range(len(en_lines)):
    en = en_lines[i]
    ro = ro_lines[i]
    ro = "[start] " + ro + " [end]"
    text_pairs.append((en, ro))

Here's what our sentence pairs look like:

In [4]:
for _ in range(5):
    print(random.choice(text_pairs))

('However, I believe that there is a great deal of untapped potential for boosting trade between both regions.', '[start] Consider, însă, că există un mare potenţial nevalorificat de creştere a schimburilor comerciale dintre cele două regiuni. [end]')
('When, for example, a constitutional court has no jurisdiction in budgetary and taxation matters, this constitutes a restriction in the separation of powers and is unquestionably a dubious situation.', '[start] De exemplu, atunci când o curte constituţională nu are jurisdicţie în aspecte bugetare şi de fiscalitate, acest lucru reprezintă o restricţie a separării puterilor în stat şi cu, siguranţă, este o situaţie dubioasă. [end]')
('In fact, the difference in prices between the beginning and the end of the food supply chain can be as high as a one to five ratio and, even if the liberals still refuse to admit it, the problems of the market need tackling to ensure reasonable prices for consumers and decent revenues for farmers.', '[start] 

Now, let's split the sentence pairs into a training set, a validation set,
and a test set.

In [5]:
random.Random(4).shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

399375 total pairs
279563 training pairs
59906 validation pairs
59906 test pairs


## Vectorizing the text data

We'll use two instances of the `TextVectorization` layer to vectorize the text
data (one for English and one for Romanian),
that is to say, to turn the original strings into integer sequences
where each integer represents the index of a word in a vocabulary.

The English layer will use the default string standardization (strip punctuation characters)
and splitting scheme (split on whitespace).

In [6]:
# strip_chars = string.punctuation
# strip_chars = strip_chars.replace("[", "")
# strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


# def custom_standardization(input_string):
#     lowercase = tf.strings.lower(input_string)
#     return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
ro_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length+1,
)
# ro_vectorization = TextVectorization(
#     max_tokens=vocab_size,
#     output_mode="int",
#     output_sequence_length=sequence_length + 1,
#     standardize=custom_standardization,
# )
train_eng_texts = [pair[0] for pair in train_pairs]
train_ro_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
ro_vectorization.adapt(train_ro_texts)

In [7]:
# print(eng_vectorization.get_vocabulary())
print(ro_vectorization.get_vocabulary())

['', '[UNK]', 'de', 'start', 'end', 'în', 'a', 'să', 'şi', 'pentru', 'care', 'la', 'și', 'că', 'este', 'cu', 'o', 'din', 'nu', 'mai', 'pe', 'un', 'se', 'acest', 'trebuie', 'sunt', 'ce', 'În', 'ar', 'fi', 'al', 'au', 'ca', 'va', 'am', 'această', 'fost', 'prin', 'privind', 'ue', 'ale', 'european', 'europene', 'lucru', 'europeană', 'toate', 'sau', 'ne', 'membre', 'fie', 'ceea', 'trebui', 'foarte', 'dacă', 'asemenea', 'aceste', 'dintre', 'mod', 'acestea', 'comisiei', 'cum', 'vor', 'comisia', 'faptul', 'statele', 'sa', 'dori', 'aceasta', 'dle', 'unei', 'le', 'mult', 'asupra', 'uniunea', 'dar', 'astfel', 'europa', 'poate', 'urmare', 'cele', 'noi', 'există', 'uniunii', 'cadrul', 'important', 'parlamentul', 'acord', 'deoarece', 'unui', 'multe', 'avem', 'acum', 'după', 'preşedinte', 'cred', 'timp', 'iar', 'special', 'acesta', 'privire', 'când', 'doar', 'precum', 'către', 'mare', 'muncă', 'aş', 'despre', 'putea', 'cât', 'vedere', 'vă', 'pot', 'acestui', 'între', 'vom', 'raport', 'are', 'lor', 'î

Next, we'll format our datasets.

At each training step, the model will seek to predict target words N+1 (and beyond)
using the source sentence and the target words 0 to N.

As such, the training dataset will yield a tuple `(inputs, targets)`, where:

- `inputs` is a dictionary with the keys `encoder_inputs` and `decoder_inputs`.
`encoder_inputs` is the vectorized source sentence and `encoder_inputs` is the target sentence "so far",
that is to say, the words 0 to N used to predict word N+1 (and beyond) in the target sentence.
- `target` is the target sentence offset by one step:
it provides the next words in the target sentence -- what the model will try to predict.

In [8]:
def format_dataset(eng, ro):
    eng = eng_vectorization(eng)
    ro = ro_vectorization(ro)
    return ({"encoder_inputs": eng, "decoder_inputs": ro[:, :-1],}, ro[:, 1:])


def make_dataset(pairs):
    eng_texts, ro_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ro_texts = list(ro_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ro_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Let's take a quick look at the sequence shapes
(we have batches of 64 pairs, and all sequences are 20 steps long):

In [9]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


## Building the model

In [10]:

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)


Next, we assemble the end-to-end model.

In [11]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [12]:
# from transformers import MarianTokenizer, AutoModelForSeq2SeqLM

# text = 'Рада познакомиться'
# mname = 'Helsinki-NLP/opus-mt-ru-en'
# tokenizer = MarianTokenizer.from_pretrained(mname)
# model = AutoModelForSeq2SeqLM.from_pretrained(mname)

## Training our model

We'll use accuracy as a quick way to monitor training progress on the validation data.

In [13]:
epochs = 40  # This should be at least 30 for convergence

checkpoint_filepath = 'transformer.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

tensorboard_callback = tf.keras.callbacks.TensorBoard()

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[model_checkpoint_callback, tensorboard_callback])

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
positional_embedding (Positiona (None, None, 256)    3845120     encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
transformer_encoder (Transforme (None, None, 256)    3155456     positional_embedding[0][0]       
________________________________________________________________________________________

## Decoding test sentences

Finally, let's demonstrate how to translate brand new English sentences.
We simply feed into the model the vectorized English sentence
as well as the target token `"[start]"`, then we repeatedly generated the next token, until
we hit the token `"[end]"`.

In [14]:
ro_vocab = ro_vectorization.get_vocabulary()
ro_index_lookup = dict(zip(range(len(ro_vocab)), ro_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = ro_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = ro_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence, translated)

Clearly, the crazy world of finance is today speculating on the food markets, without ever wanting to touch the goods themselves, which are constantly bought and sold. [start] În mod evident lumea [UNK] a [UNK] astăzi pe pieţele alimentare fără a se dori să [UNK] produsele [UNK] în
This standard also needs to look to the future. [start] acest standard trebuie să [UNK] viitorul end             
It is unfortunate, however, that the motion also stands behind the Basel II provisions which, we know from experience, have led to the flow of capital to small and medium-sized businesses practically running dry. [start] cu toate acestea este regretabil faptul că propunerea se află în spatele [UNK] de fapt acest lucru este regretabil că
Rather, investing in a greater number of cross-border connections would more effectively reduce dependence on one or more third countries in the long term. [start] mai degrabă să [UNK] o serie de [UNK] transfrontaliere ar reduce mai mult dependenţa de un ţară sau 

In [ ]:
test_eng_texts = [pair[0] for pair in test_pairs]
test_ro_texts = [pair[1] for pair in test_pairs]
pred_ro_texts = []
# tqdm._instances.clear()
for i in tqdm(range(len(test_eng_texts))):
    pred_ro_text = decode_sequence(test_eng_texts[i])
    pred_ro_texts.append(pred_ro_text)

score = corpus_bleu(test_ro_texts, pred_ro_texts)
print(score)

 89%|████████▉ | 53355/59906 [3:36:18<26:51,  4.07it/s]  